In [ ]:
import itertools
import os

def generate_plumed_input_manual(atom_mapping, output_dir="path/to/output"):
    """
    Generate PLUMED input using fully manual atom indices.
    
    Args:
        atom_mapping (dict): {generic_number: atom_index}
        output_dir (str): Where to save the PLUMED file
    """
    os.makedirs(output_dir, exist_ok=True)
    plumed_file_path = os.path.join(output_dir, "distances.dat")
    
    lines = ["# Define distances between relevant atom pairs"]
    counter = 0
    for it, (gen1, gen2) in enumerate(itertools.combinations(sorted(atom_mapping.keys()), 2), start=1):
        atom1_index = atom_mapping[gen1]
        atom2_index = atom_mapping[gen2]
        lines.append(f"d{it}: DISTANCE ATOMS={atom1_index},{atom2_index}  # {gen1} - {gen2}")
        counter += 1

    if counter > 0:
        distances = [f"d{i}" for i in range(1, counter + 1)]
        lines.extend([
            "# Print values every 12500 steps, every 25 ps",
            f"PRINT FILE=distances STRIDE=12500 ARG={','.join(distances)}"
        ])

    with open(plumed_file_path, "w") as f:
        for line in lines:
            f.write(line + "\n")

    print(f"PLUMED input file written to: {plumed_file_path}")

def parse_pdb_for_ca_mapping(pdb_path):
    mapping = {}

    with open(pdb_path, "r") as f:
        for line in f:
            if not line.startswith("ATOM"):
                continue

            atom_name = line[12:16].strip()
            res_name = line[17:20].strip()
            chain_or_segid = line[72:76].strip()  
            res_num = line[22:26].strip()
            atom_num = int(line[6:11])

            if atom_name == "CA":
                key = f"{chain_or_segid}_{res_name}{res_num}"
                mapping[key] = atom_num

    return mapping


    

if __name__ == "__main__":
    pdb_file = "path/to/step5_input.pdb"  #for charmm gui step5_input.pdb
    ca_mapping = parse_pdb_for_ca_mapping(pdb_file)
    for k, v in ca_mapping.items():
        print(f'"{k}": {v},')
    generate_plumed_input_manual(ca_mapping)

    


In [ ]:
import re
import os
from collections import OrderedDict

def parse_plumed_file(file_path):
    """
    Parse a PLUMED distances.dat file and extract distance definitions.
    
    Returns:
        list: [(d_number, atom1, atom2, label1, label2, full_line), ...]
    """
    distances = []
    other_lines = []
    
    with open(file_path, 'r') as f:
        for line in f:
            # Match lines like: d1: DISTANCE ATOMS=1234,5678  # LABEL1 - LABEL2 
            match = re.match(r'd(\d+):\s*DISTANCE\s+ATOMS=(\d+),(\d+)\s*#\s*(.+?)\s*-\s*(.+)', line)
            if match:
                d_num = int(match.group(1))
                atom1 = int(match.group(2))
                atom2 = int(match.group(3))
                label1 = match.group(4).strip()
                label2 = match.group(5).strip()
                
                distances.append((d_num, atom1, atom2, label1, label2, line))
            else:
                other_lines.append(line)
    
    return distances, other_lines

def create_canonical_ordering(all_files_data):
    """
    Create a canonical ordering of all unique pairs across all files.
    Uses alphabetical sorting of label pairs.
    
    Args:
        all_files_data: dict of {filename: (distances_list, other_lines)}
    
    Returns:
        OrderedDict: {(label1, label2): new_d_number}
    """
    # Collect all unique pairs
    all_pairs = set()
    
    for distances, _ in all_files_data.values():
        for _, atom1, atom2, label1, label2, _ in distances:
            # Normalize the pair (alphabetically)
            pair = tuple(sorted([label1, label2]))
            all_pairs.add(pair)
    
    # Sort pairs alphabetically for consistent ordering
    sorted_pairs = sorted(all_pairs)
    
    # Create mapping with new d-numbers
    canonical_mapping = OrderedDict()
    for i, pair in enumerate(sorted_pairs, start=1):
        canonical_mapping[pair] = i
    
    return canonical_mapping

def renumber_file(file_path, canonical_mapping, output_path=None):
    """
    Renumber a PLUMED file according to canonical mapping.
    
    Args:
        file_path: Path to original file
        canonical_mapping: OrderedDict mapping (label1, label2) -> new_d_number
        output_path: Where to save renumbered file (None = overwrite original)
    """
    distances, other_lines = parse_plumed_file(file_path)
    
    new_distances = []
    
    for old_d_num, atom1, atom2, label1, label2, old_line in distances:
        # Normalize pair for lookup
        pair = tuple(sorted([label1, label2]))
        new_d_num = canonical_mapping[pair]
        
        # Reconstruct the line with new d-number
        new_line = f"d{new_d_num}: DISTANCE ATOMS={atom1},{atom2}  # {label1} - {label2}\n"
        new_distances.append((new_d_num, new_line))
    
    # Sort by new d-number
    new_distances.sort(key=lambda x: x[0])
    
    if output_path is None:
        output_path = file_path
    
    with open(output_path, 'w') as f:
        for line in other_lines:
            if line.strip().startswith('#') or not line.strip():
                f.write(line)
                if 'Define distances' in line:
                    break
        
        for _, line in new_distances:
            f.write(line)
        
        total_distances = len(new_distances)
        if total_distances > 0:
            distance_list = ','.join([f"d{i}" for i in range(1, total_distances + 1)])
            f.write("# Print values every 12500 steps, every 25 ps\n")
            f.write(f"PRINT FILE=distances STRIDE=12500 ARG={distance_list}\n")
    
    return len(new_distances)

def fix_plumed_files(file_paths, create_backup=True):
    """
    Renumber all PLUMED files to have consistent d-number assignments.
    
    Args:
        file_paths: List of file paths to fix
        create_backup: If True, create .bak files before modifying
    """
    print("="*80)
    print("PLUMED FILE RENUMBERING SCRIPT")
    print("="*80)
    
    all_files_data = {}
    for path in file_paths:
        if not os.path.exists(path):
            print(f"WARNING: File {path} does not exist, skipping...")
            continue
        
        print(f"\nReading: {path}")
        distances, other_lines = parse_plumed_file(path)
        print(f"  Found {len(distances)} distance definitions")
        all_files_data[path] = (distances, other_lines)
    
    if not all_files_data:
        print("\nERROR: No valid files to process!")
        return
    
    # Create canonical ordering
    print("\n" + "="*80)
    print("Creating canonical ordering...")
    canonical_mapping = create_canonical_ordering(all_files_data)
    print(f"Total unique pairs: {len(canonical_mapping)}")
    
    # For quick checking of outputs 
    print("\nExample mappings:")
    for i, (pair, d_num) in enumerate(list(canonical_mapping.items())[:5]):
        print(f"  d{d_num}: {pair[0]} - {pair[1]}")
    print("  ...")
    
    print("\n" + "="*80)
    print("Renumbering files...")
    print("="*80)
    
    for file_path in all_files_data.keys():
        if create_backup:
            backup_path = file_path + ".bak"
            print(f"\nCreating backup: {backup_path}")
            
            with open(file_path, 'r') as f_in:
                with open(backup_path, 'w') as f_out:
                    f_out.write(f_in.read())
        
        print(f"Renumbering: {file_path}")
        num_distances = renumber_file(file_path, canonical_mapping)
        print(f"  Wrote {num_distances} renumbered distance definitions")
    
    print("\n" + "="*80)
    print("RENUMBERING COMPLETE!")
    print("="*80)
    if create_backup:
        print("\nOriginal files backed up with .bak extension")

if __name__ == "__main__":
    import sys
    
    # Filter out Jupyter kernel arguments
    filtered_args = [arg for arg in sys.argv[1:] if not arg.startswith('--') and not arg.startswith('-f=')]
    
    if len(filtered_args) > 0:
        file_paths = filtered_args
    else:
        file_paths = [
            "path/to example/GABA_open/gromacs/distances.dat",
            "path/to example/Bicuculline_resting/gromacs/distances.dat",
            "path/to example/Picrotoxin_desensitized/gromacs/distances.dat"
            "path/to example/Phenobarbital_GABA_open/gromacs/distances.dat"
        ]
    # keep backup 
    fix_plumed_files(file_paths, create_backup=True)